<a href="https://colab.research.google.com/github/moveformyfuture/House_Price_Expectation/blob/main/House_Price_Expectation_Regression(ETR%2BRF%2BGBR%2BXGB%2BHGB%2BNGB%2BCAT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 머신러닝을 위한 모듈 설치
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install sklearn
!pip3 install catboost
!pip3 install ngboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 84 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 349 kB 12.7 MB/s 
     |████████████████████████████████| 68 kB 7.5 MB/s 
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4048 sha256=64106ba80a0824b20c096cac333e4de78564

In [2]:
import numpy as np
import pandas as pd
from pandas import read_csv, set_option

from sklearn.model_selection import cross_val_score, train_test_split, RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.utils import resample, shuffle

# 구글 드라이브에 접근할 수 있도록 아래 코드 입력
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 경고 무시
import warnings
warnings.filterwarnings('ignore')

In [5]:
# 데이터셋 설정
X_train_df=read_csv('/content/drive/MyDrive/데이터분석 프로젝트 자료/집값예측 프로젝트/train.csv')
X_test_df=read_csv('/content/drive/MyDrive/데이터분석 프로젝트 자료/집값예측 프로젝트/test.csv')

In [ ]:
# 데이터셋 확인(train : traget가 존재)
X_train_df.describe()

In [ ]:
# 데이터셋 확인(test : target 열이 없음)
X_test_df.describe()

In [ ]:
X_train_df[X_train_df['Garage Yr Blt']>=2022]

,id,Overall Qual,Gr Liv Area,Exter Qual,Garage Cars,Garage Area,Kitchen Qual,Total Bsmt SF,1st Flr SF,Bsmt Qual,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt,target
254,255,8,1564,Gd,2,502,Ex,1546,1564,Gd,2,2006,2007,2207,267300


In [ ]:
X_train_df[X_train_df['Year Built']==2006]

,id,Overall Qual,Gr Liv Area,Exter Qual,Garage Cars,Garage Area,Kitchen Qual,Total Bsmt SF,1st Flr SF,Bsmt Qual,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt,target
1,2,7,1352,Gd,2,466,Gd,1352,1352,Ex,2,2006,2007,2006,194000
19,20,6,1326,Gd,2,427,Gd,1326,1326,Gd,2,2006,2006,2006,174190
67,68,8,1460,Gd,2,480,Gd,1460,1460,Ex,2,2006,2007,2006,247000
97,98,6,1495,Gd,2,440,Gd,738,738,Gd,2,2006,2006,2006,171925
102,103,8,1324,Gd,2,550,Gd,1324,1324,Gd,2,2006,2006,2006,254000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1288,1289,7,1977,Gd,3,574,Gd,982,982,Gd,2,2006,2006,2006,250000
1327,1328,8,1634,Gd,3,866,Gd,1626,1634,Gd,2,2006,2006,2006,250000
1328,1329,8,1370,Gd,2,484,Gd,1370,1370,Gd,2,2006,2006,2006,205000
1336,1337,10,2698,Ex,3,736,Ex,1850,1850,Ex,2,2006,2006,2006,535000


In [ ]:
# train 데이터 수정 (2207 비정상적 숫자를 2006으로 변경)
X_train_df.loc[(X_train_df['Garage Yr Blt'] == 2207), 'Garage Yr Blt']=2006
#df.loc(row, col) : df에 대한 인덱싱

In [ ]:
# 범주 형식의 데이터를 숫자로 변환(Label-인코딩 방법)
# 'X_train' 데이터를 인코딩
X_train_df=X_train_df.copy()
X_train_df[X_train_df.columns[X_train_df.dtypes=='O']] = X_train_df[X_train_df.columns[X_train_df.dtypes=='O']].astype(str).apply(LabelEncoder().fit_transform)
# astype() : 데이터 타입 바꾸기, LabelEncoder() : 라벨인코딩
# 데이터 타입이 object인 columns만 선택해서 indexing = str로 데이터 타입을 바꾼 후 LabelEncoder() 함수를 적용
X_train_df.info() # 데이터 변경 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   id              1350 non-null   int64
 1   Overall Qual    1350 non-null   int64
 2   Gr Liv Area     1350 non-null   int64
 3   Exter Qual      1350 non-null   int64
 4   Garage Cars     1350 non-null   int64
 5   Garage Area     1350 non-null   int64
 6   Kitchen Qual    1350 non-null   int64
 7   Total Bsmt SF   1350 non-null   int64
 8   1st Flr SF      1350 non-null   int64
 9   Bsmt Qual       1350 non-null   int64
 10  Full Bath       1350 non-null   int64
 11  Year Built      1350 non-null   int64
 12  Year Remod/Add  1350 non-null   int64
 13  Garage Yr Blt   1350 non-null   int64
 14  target          1350 non-null   int64
dtypes: int64(15)
memory usage: 158.3 KB


In [ ]:
# 범주 형식의 데이터를 숫자로 변환(Label-인코딩 방법)
# 'X_test' 데이터를 인코딩
X_test_df=X_test_df.copy()
X_test_df[X_test_df.columns[X_test_df.dtypes=='O']] = X_test_df[X_test_df.columns[X_test_df.dtypes=='O']].astype(str).apply(LabelEncoder().fit_transform)
# astype() : 데이터 타입 바꾸기, LabelEncoder() : 라벨인코딩
# 데이터 타입이 object인 columns만 선택해서 indexing = str로 데이터 타입을 바꾼 후 LabelEncoder() 함수를 적용
X_test_df.info() # 데이터 변경 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   id              1350 non-null   int64
 1   Overall Qual    1350 non-null   int64
 2   Gr Liv Area     1350 non-null   int64
 3   Exter Qual      1350 non-null   int64
 4   Garage Cars     1350 non-null   int64
 5   Garage Area     1350 non-null   int64
 6   Kitchen Qual    1350 non-null   int64
 7   Total Bsmt SF   1350 non-null   int64
 8   1st Flr SF      1350 non-null   int64
 9   Bsmt Qual       1350 non-null   int64
 10  Full Bath       1350 non-null   int64
 11  Year Built      1350 non-null   int64
 12  Year Remod/Add  1350 non-null   int64
 13  Garage Yr Blt   1350 non-null   int64
dtypes: int64(14)
memory usage: 147.8 KB


In [ ]:
# X, y에 대한 설정
X=X_train_df.drop(['id', 'target'], axis=1) # 불필요 변수 제거
y=X_train_df.target # y에는 target 열만 나오도록 설정
X_test_forc = X_test_df.drop(['id'], axis=1) # K-fold 이전의 catboost 값 도출 용도
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   Overall Qual    1350 non-null   int64
 1   Gr Liv Area     1350 non-null   int64
 2   Exter Qual      1350 non-null   int64
 3   Garage Cars     1350 non-null   int64
 4   Garage Area     1350 non-null   int64
 5   Kitchen Qual    1350 non-null   int64
 6   Total Bsmt SF   1350 non-null   int64
 7   1st Flr SF      1350 non-null   int64
 8   Bsmt Qual       1350 non-null   int64
 9   Full Bath       1350 non-null   int64
 10  Year Built      1350 non-null   int64
 11  Year Remod/Add  1350 non-null   int64
 12  Garage Yr Blt   1350 non-null   int64
dtypes: int64(13)
memory usage: 137.2 KB


In [ ]:
# 파생변수 생성
# 변수 선택을 위해 후진제거법 사용
# 후진제거법 : 모든 변수가 포함된 모형에서 불필요한 변수들을 제거해 나가는 과정 반복
# 결론 : 모든 변수를 활용할 때 가장 좋은 성능을 보임
def feature(data):
  df = data.copy()
  df['Year Gap Remod'] = df['Year Remod/Add'] - df['Year Built'] # Year Gap Remod(연도 차이) = '리모델링 연도' - '완공 연도'
  df['Car Area'] = df['Garage Area']/df['Garage Cars'] # Car Area (1대당 차고 사용 면적) ='차고 면적' / '차고 자리 개수'
  df['2nd flr SF'] = df['Gr Liv Area'] - df['1st Flr SF'] # '2nd flr SF(2층 면적)' = '지상층 생활 면적' - '1층 면적'
  df['2nd flr'] = df['2nd flr SF'].apply(lambda x : 1 if x > 0 else 0) # '2nd flr(2층 유무 확인)' = 2층 면적이 0이면 0, 그 외이면 1 입력
  df['Total SF'] = df[['Gr Liv Area',"Garage Area", "Total Bsmt SF"]].sum(axis=1) # Total SF(총 면적) = '지상층 생활 면적' + '차고 면적' + '지하실 면적'
  df['Sum Qual'] = df[["Exter Qual", "Kitchen Qual", "Overall Qual"]].sum(axis=1) # Sum Qual(총 품질) = '외관 재료 품질' + '부엌 품질' + '전반적 재료와 마감 품질'
  df['Garage InOut'] = df.apply(lambda x : 1 if x['Gr Liv Area'] != x['1st Flr SF'] else 0, axis=1) # Garage InO(ut차고 위치 확인) = '지상층 생활 면적' = '1층 면적' 이면 1, 아니면 0
  return df

In [ ]:
# X에 파생 변수 추가
X=feature(X)
X_test_c = feature(X_test_forc) # K-fold 이전의 catboost 값 도출 용도
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Overall Qual    1350 non-null   int64  
 1   Gr Liv Area     1350 non-null   int64  
 2   Exter Qual      1350 non-null   int64  
 3   Garage Cars     1350 non-null   int64  
 4   Garage Area     1350 non-null   int64  
 5   Kitchen Qual    1350 non-null   int64  
 6   Total Bsmt SF   1350 non-null   int64  
 7   1st Flr SF      1350 non-null   int64  
 8   Bsmt Qual       1350 non-null   int64  
 9   Full Bath       1350 non-null   int64  
 10  Year Built      1350 non-null   int64  
 11  Year Remod/Add  1350 non-null   int64  
 12  Garage Yr Blt   1350 non-null   int64  
 13  Year Gap Remod  1350 non-null   int64  
 14  Car Area        1350 non-null   float64
 15  2nd flr SF      1350 non-null   int64  
 16  2nd flr         1350 non-null   int64  
 17  Total SF        1350 non-null   i

In [ ]:
#  학습셋과 검증셋 나누기
X_train, X_valid, y_train, y_valid=train_test_split(X,y,test_size=0.2, random_state=0)

In [ ]:
# 점수 채점식 생성
from sklearn.metrics import make_scorer

def NMAE(true, pred):
    mae = np.mean(np.abs(true - pred))
    score = mae / np.mean(np.abs(true))
    return score

nmae_score = make_scorer(NMAE, greater_is_better=False) # make_scorer(함수명, greater_is_better=Boolean)
                                                        # greater_is_better : 높은게 좋은지, 낮은게 좋은지 표시

In [ ]:
# 회귀 모델링 불러오기
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import AdaBoostRegressor
from catboost import CatBoostRegressor, Pool
from ngboost import NGBRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet

In [ ]:
# 회귀 모델링 정의(하이퍼 파라미터 조정)
etr=ExtraTreesRegressor(n_estimators=200)
lr=LinearRegression()
rg=Ridge()
rf=RandomForestRegressor(random_state = 42, criterion = 'mae')
gbr=GradientBoostingRegressor(random_state = 42, max_depth = 4, learning_rate=0.09, loss='huber', n_estimators=130)
xgb=XGBRegressor(random_state = 42, learning_rate=0.05, n_estimators=200)
lgb=LGBMRegressor(random_state = 42, learning_rate=0.05, n_estimators=200)
ada=AdaBoostRegressor(random_state = 42, learning_rate=0.03, n_estimators=100)
hgb=HistGradientBoostingRegressor(random_state=42, scoring=nmae_score, learning_rate=0.09)
ngb=NGBRegressor(random_state = 42, n_estimators = 500, verbose = 0, learning_rate = 0.03)
cat_reg=CatBoostRegressor(depth = 4, random_state = 42, loss_function = 'MAE', n_estimators = 3000, learning_rate = 0.03, verbose = 0)

In [ ]:
# 데이터 학습 : ML.fit(X_train, y_train)
etr.fit(X_train,y_train)
lr.fit(X_train,y_train)
rg.fit(X_train,y_train)
rf.fit(X_train,y_train)
gbr.fit(X_train,y_train)
xgb.fit(X_train,y_train)
lgb.fit(X_train,y_train)
ada.fit(X_train,y_train)
hgb.fit(X_train,y_train)
ngb.fit(X_train,y_train)
cat_reg.fit(X_train,y_train)

[14:48:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
# 데이터 검증 예측값 저장
y_etr=etr.predict(X_valid)
y_lr=lr.predict(X_valid)
y_rg=rg.predict(X_valid)
y_rf=rf.predict(X_valid)
y_gbr=gbr.predict(X_valid)
y_xgb=xgb.predict(X_valid)
y_lgb=lgb.predict(X_valid)
y_ada=ada.predict(X_valid)
y_hgb=hgb.predict(X_valid)
y_ngb=ngb.predict(X_valid)
y_cat=cat_reg.predict(X_valid)

In [ ]:
# 평가지표에 삽입 - 실제값과 예측값 비교
score_etr=NMAE(y_valid, y_etr)
score_lr=NMAE(y_valid, y_lr)
score_rg=NMAE(y_valid, y_rg)
score_rf=NMAE(y_valid, y_rf)
score_gbr=NMAE(y_valid, y_gbr)
score_xgb=NMAE(y_valid, y_xgb)
score_lgb=NMAE(y_valid, y_lgb)
score_ada=NMAE(y_valid, y_ada)
score_hgb=NMAE(y_valid, y_hgb)
score_ngb=NMAE(y_valid, y_ngb)
score_cat=NMAE(y_valid, y_cat)

In [ ]:
# 평가지표 점수 출력
print(score_etr)
print(score_lr)
print(score_rg)
print(score_rf)
print(score_gbr)
print(score_xgb)
print(score_lgb)
print(score_ada)
print(score_hgb)
print(score_ngb)
print(score_cat)
# 여기까지 단순 Regression

0.09756234520300468
0.11786844281723506
0.11781620420650744
0.09754354996249787
0.09408704494320543
0.09617031838709376
0.1017267128731998
0.11660261782730845
0.10239431439002132
0.09705199647856395
0.09347795898855675


In [ ]:
# KFold 활용해서 fold별 val, predict 값 평균내어 일반화 성능을 끌어올림
from sklearn.model_selection import KFold
kf = KFold(n_splits = 10, random_state = 42, shuffle = True) # n_splits : K=10으로 설정을 의미, shuffle=분할 전 섞음을 의미

In [ ]:
# Kfold 활용을 위해 데이터셋을 다시 정의
# Kfold : 전체 데이터(학습+검증)을 k개의 데이터셋으로 나누어 교차검증 수행

X = X_train_df.drop(['id', 'target'], axis=1)
X_test = X_test_df.drop(['id'], axis=1)

In [ ]:
# 파생변수 정의 (앞과 동일)
def feature(data):
  df = data.copy()
  df['Year Gap Remod'] = df['Year Remod/Add'] - df['Year Built']
  df['Car Area'] = df['Garage Area']/df['Garage Cars']
  df['2nd flr SF'] = df['Gr Liv Area'] - df['1st Flr SF']
  df['2nd flr'] = df['2nd flr SF'].apply(lambda x : 1 if x > 0 else 0)
  df['Total SF'] = df[['Gr Liv Area',"Garage Area", "Total Bsmt SF"]].sum(axis=1)
  df['Sum Qual'] = df[["Exter Qual", "Kitchen Qual", "Overall Qual"]].sum(axis=1)
  df['Garage InOut'] = df.apply(lambda x : 1 if x['Gr Liv Area'] != x['1st Flr SF'] else 0, axis=1)
  return df

In [ ]:
# X에 파생변수 추가
X = feature(X)
y = np.log1p(X_train_df.target) # y는 traget 열만 가져오고, 데이터를 고르게 분포하도록 만들기 위해 로그를 취함
target = feature(X_test) # X_test_df 에서 'id' 삭제, 파생변수 추가
target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Overall Qual    1350 non-null   int64  
 1   Gr Liv Area     1350 non-null   int64  
 2   Exter Qual      1350 non-null   int64  
 3   Garage Cars     1350 non-null   int64  
 4   Garage Area     1350 non-null   int64  
 5   Kitchen Qual    1350 non-null   int64  
 6   Total Bsmt SF   1350 non-null   int64  
 7   1st Flr SF      1350 non-null   int64  
 8   Bsmt Qual       1350 non-null   int64  
 9   Full Bath       1350 non-null   int64  
 10  Year Built      1350 non-null   int64  
 11  Year Remod/Add  1350 non-null   int64  
 12  Garage Yr Blt   1350 non-null   int64  
 13  Year Gap Remod  1350 non-null   int64  
 14  Car Area        1350 non-null   float64
 15  2nd flr SF      1350 non-null   int64  
 16  2nd flr         1350 non-null   int64  
 17  Total SF        1350 non-null   i

In [ ]:
# k개 fold별 NMAE점수 평균계산
# ExtraTreesRegression
etr_pred = np.zeros(target.shape[0]) # etr_pred = 1350개의 행을 0으로 반환
                                     # zeros(shape, dtype, order) : 0으로 된 matrix 생성
                                     # shape[0] : 전체 행의 수를 반환
etr_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) : # enumerate : index와 해당 원소를 tuple() 형태로 반환
                                                        # kf.split(X, y) : X, y를 인덱스로 추출해 각각 tr_idx, val_idx에 저장
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx]) # val_y에는 다시 지수함수를 취해 원래값을 반환
    
    etr = ExtraTreesRegressor(n_estimators=200) # 회귀모델링을 정의
    etr.fit(tr_x, tr_y) # (tr_x, tr_y)를 etr에 넣어 학습
    
    val_pred = np.expm1(etr.predict(val_x)) # val_x를 넣어 예측한 값에 지수를 취해서 저장(log를 취한 y로 예측했기 때문에 값의 분포가 log로 나올 것이기 때문)
    val_nmae = NMAE(val_y, val_pred) # val_y(실제)와 val_pred(예측) 비교
    etr_val.append(val_nmae) # 리스트로 만듦
    print(f'{n + 1} FOLD NMAE = {val_nmae}') # NMAE점수 출력
    
    target_data = Pool(data = target, label = None)
    fold_pred = etr.predict(target) / 10 # target 데이터로 예측한 값을 10으로 나눠줌 (평균 내기 위해)
    etr_pred += fold_pred # 해당 값을 초기에 만들어준 0행렬에 지속 더해줌 --> 예측한 값의 평균 행렬 생성
print(f'10 FOLD Mean of NMAE = {np.mean(etr_val)}')

1 FOLD NMAE = 0.08082294083501068
2 FOLD NMAE = 0.09805282758023898
3 FOLD NMAE = 0.09820825774942313
4 FOLD NMAE = 0.11368983534171043
5 FOLD NMAE = 0.08917934122108033
6 FOLD NMAE = 0.09280248811398317
7 FOLD NMAE = 0.08670929904797334
8 FOLD NMAE = 0.09394763344839425
9 FOLD NMAE = 0.09771716552163262
10 FOLD NMAE = 0.1049802350823752
10 FOLD Mean of NMAE = 0.09561100239418222


In [ ]:
# RandomForestRegressor
rf_pred = np.zeros(target.shape[0])
rf_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    rf = RandomForestRegressor(random_state = 42, criterion = 'mae')
    rf.fit(tr_x, tr_y)
    
    val_pred = np.expm1(rf.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    rf_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}')
    
    fold_pred = rf.predict(target) / 10
    rf_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(rf_val)}')

1 FOLD NMAE = 0.08644939652170183
2 FOLD NMAE = 0.09802122981071265
3 FOLD NMAE = 0.09441514507875699
4 FOLD NMAE = 0.11919078028347765
5 FOLD NMAE = 0.09300279322558394
6 FOLD NMAE = 0.09380141798754157
7 FOLD NMAE = 0.09524411974642773
8 FOLD NMAE = 0.09621723726204115
9 FOLD NMAE = 0.09727894979270606
10 FOLD NMAE = 0.10679952064531323
10FOLD Mean of NMAE = 0.09804205903542629


In [ ]:
# GradientBoostingRegressor
gbr_pred = np.zeros(target.shape[0])
gbr_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    gbr = GradientBoostingRegressor(random_state = 42, max_depth = 4, learning_rate=0.09, loss='huber', n_estimators=130)
    gbr.fit(tr_x, tr_y)
    
    val_pred = np.expm1(gbr.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    gbr_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}')
    
    fold_pred = gbr.predict(target) / 10
    gbr_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(gbr_val)}')

1 FOLD NMAE = 0.07750171187548267
2 FOLD NMAE = 0.10402255744236638
3 FOLD NMAE = 0.09909082590440864
4 FOLD NMAE = 0.11045415796367825
5 FOLD NMAE = 0.08593246978859838
6 FOLD NMAE = 0.09825484733485763
7 FOLD NMAE = 0.09720505729733409
8 FOLD NMAE = 0.0948004465481032
9 FOLD NMAE = 0.09141972579762334
10 FOLD NMAE = 0.10555228683710073
10FOLD Mean of NMAE = 0.09642340867895534


In [ ]:
# xgbr
xgb_pred = np.zeros(target.shape[0])
xgb_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    xgb = XGBRegressor(random_state = 42, learning_rate=0.05, n_estimators=200)
    xgb.fit(tr_x, tr_y)
    
    val_pred = np.expm1(xgb.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    xgb_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}')
    
    fold_pred = xgb.predict(target) / 10
    xgb_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(xgb_val)}')

[14:50:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
1 FOLD NMAE = 0.08428037256373816
[14:50:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
2 FOLD NMAE = 0.10138865684839769
[14:50:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
3 FOLD NMAE = 0.0989747248109777
[14:50:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
4 FOLD NMAE = 0.11571228665447786
[14:50:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
5 FOLD NMAE = 0.08553477586229843
[14:50:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
6 FOLD NMAE = 0.10000616511188161
[14:50:37] WARNING: /workspace/src/

In [ ]:
# lgbr
lgb_pred = np.zeros(target.shape[0])
lgb_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    lgb = LGBMRegressor(random_state = 42, learning_rate=0.05, n_estimators=200)
    lgb.fit(tr_x, tr_y)
    
    val_pred = np.expm1(lgb.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    lgb_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}')
    
    fold_pred = lgb.predict(target) / 10
    lgb_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(lgb_val)}')

1 FOLD NMAE = 0.07625613915032772
2 FOLD NMAE = 0.09189015613865868
3 FOLD NMAE = 0.09269203870995357
4 FOLD NMAE = 0.11143669532187271
5 FOLD NMAE = 0.09217377021357274
6 FOLD NMAE = 0.0977746884597186
7 FOLD NMAE = 0.08801058220816765
8 FOLD NMAE = 0.09986396458870156
9 FOLD NMAE = 0.09318176273003355
10 FOLD NMAE = 0.10790359695717838
10FOLD Mean of NMAE = 0.09511833944781851


In [ ]:
# HGBRegressor
hgbr_pred = np.zeros(target.shape[0])
hgbr_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    hgbr = HistGradientBoostingRegressor(random_state=42, scoring=nmae_score, learning_rate=0.09)
    hgbr.fit(tr_x, tr_y)
    
    val_pred = np.expm1(hgbr.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    hgbr_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}')
    
    fold_pred = hgbr.predict(target) / 10
    hgbr_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(hgbr_val)}')

1 FOLD NMAE = 0.07603103078693206
2 FOLD NMAE = 0.09723516449257738
3 FOLD NMAE = 0.09643893392723443
4 FOLD NMAE = 0.11172336623111918
5 FOLD NMAE = 0.09275907851348504
6 FOLD NMAE = 0.09621071514289606
7 FOLD NMAE = 0.08716331797336076
8 FOLD NMAE = 0.10151042043583
9 FOLD NMAE = 0.09374173702935405
10 FOLD NMAE = 0.10719969919676056
10FOLD Mean of NMAE = 0.09600134637295496


In [ ]:
# NGBRegressor
ngb_pred = np.zeros(target.shape[0])
ngb_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    ngb = NGBRegressor(random_state = 42, n_estimators = 500, verbose = 0, learning_rate = 0.03)
    ngb.fit(tr_x, tr_y, val_x, val_y, early_stopping_rounds = 300)
    
    val_pred = np.expm1(ngb.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    ngb_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}')
    
    target_data = Pool(data = target, label = None)
    fold_pred = ngb.predict(target) / 10
    ngb_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(ngb_val)}')

1 FOLD NMAE = 0.07801828661285823
2 FOLD NMAE = 0.10042493059713078
3 FOLD NMAE = 0.09792787174179249
4 FOLD NMAE = 0.11187983508636111
5 FOLD NMAE = 0.08664500737352729
6 FOLD NMAE = 0.0974549188225471
7 FOLD NMAE = 0.09678827424108716
8 FOLD NMAE = 0.09082031796536544
9 FOLD NMAE = 0.0915733487858593
10 FOLD NMAE = 0.10493894853304335
10FOLD Mean of NMAE = 0.09564717397595722


In [ ]:
# cat
cat_pred = np.zeros(target.shape[0])
cat_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    tr_data = Pool(data = tr_x, label = tr_y)
    val_data = Pool(data = val_x, label = val_y)
    
    cat = CatBoostRegressor(depth = 4, random_state = 42, loss_function = 'MAE', n_estimators = 3000, learning_rate = 0.03, verbose = 0)
    cat.fit(tr_data, eval_set = val_data, early_stopping_rounds = 750, verbose = 1000)
    
    val_pred = np.expm1(cat.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    cat_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}')
    
    target_data = Pool(data = target, label = None)
    fold_pred = cat.predict(target) / 10
    cat_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(cat_val)}')

0:	learn: 0.2938486	test: 184599.8961000	best: 184599.8961000 (0)	total: 1.45ms	remaining: 4.35s
Stopped by overfitting detector  (750 iterations wait)

bestTest = 184599.8459
bestIteration = 130

Shrink model to first 131 iterations.
1 FOLD NMAE = 0.0832515027438851
0:	learn: 0.2948005	test: 182774.3369942	best: 182774.3369942 (0)	total: 1.38ms	remaining: 4.14s
Stopped by overfitting detector  (750 iterations wait)

bestTest = 182774.3175
bestIteration = 114

Shrink model to first 115 iterations.
2 FOLD NMAE = 0.103085993983557
0:	learn: 0.2883000	test: 196831.3027016	best: 196831.3027016 (0)	total: 1.44ms	remaining: 4.31s
1000:	learn: 0.0671716	test: 196831.2177815	best: 196831.2171394 (689)	total: 1.23s	remaining: 2.46s
Stopped by overfitting detector  (750 iterations wait)

bestTest = 196831.2171
bestIteration = 689

Shrink model to first 690 iterations.
3 FOLD NMAE = 0.0941226411035661
0:	learn: 0.2912214	test: 172746.7630999	best: 172746.7630999 (0)	total: 1.37ms	remaining: 4.1s


In [ ]:
# 검증 성능 확인
val_list = [etr_val, rf_val, gbr_val, xgb_val, lgb_val, hgbr_val, ngb_val, cat_val]
for val in val_list :
  print("{:.8f}".format(np.mean(val))) 

0.09561100
0.09804206
0.09642341
0.09803039
0.09511834
0.09600135
0.09564717
0.12374755


In [ ]:
# catboost의 성능이 좋지 않아, k-fold 이전의 catboost를 불러오기
predict_cat=cat_reg.predict(X_test_c)
print(predict_cat)

[333636.50168837 126244.31752535 175668.56762463 ...  67941.58672409
 196782.1214807  140186.52526413]


In [ ]:
# 제출용 파일 불러오기.
submission = pd.read_csv('/content/drive/MyDrive/DACON 데이터분석 대회/집값예측 프로젝트/submission.csv')
submission.head()

,id,target
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [ ]:
# exp를 취해 cat과의 상태를 맞춘 후 산술평균으로 결과 도출
submission['target'] = (np.expm1(etr_pred) + np.expm1(rf_pred) + np.expm1(gbr_pred) + np.expm1(xgb_pred) + np.expm1(hgbr_pred) + np.expm1(ngb_pred) + predict_cat) / 7
submission

,id,target
0,1,335784.217516
1,2,126447.570503
2,3,174508.218216
3,4,251946.632531
4,5,134058.007874
...,...,...
1345,1346,322518.634052
1346,1347,125689.608234
1347,1348,84617.252298
1348,1349,195025.224852


In [ ]:
submission.to_csv("submission.csv", index=False)